# Machine Learning - Final assignment

**Students:**
<hr>
Mutaz Abueisheh</br>
Marcelo Jose Ferrer</br>
Maximiliano Hormazábal Lagos</br>
Mohamed Aymen Merchaoui</br>

## Classification problem

Classify the dataset between this 10 classes.

0 = Acoustic/Folk</br>
1 = Alternative music</br>
2 = Blues</br>
3 = Bollywood</br>
4 = Country</br>
5 = Hip Hop</br>
6 = Indie</br>
7 = Instrumental</br>
8 = Metal</br>
9 = Pop</br>
10 = Rock</br>

https://www.kaggle.com/datasets/purumalgi/music-genre-classification

# Imports and declarations

This section contains all imports and declarations

In [1]:
# The next packages must be installed to run the solution
import Pkg; 
#Pkg.add("Flux")
#Pkg.add("RDatasets")
#Pkg.add("FeatureSelectors")
#Pkg.add("ScikitLearn"))
#Pkg.add("WeightedPCA"))
#Pkg.add("BetaML")
# Packages used To store and load models in and from disk
# Pkg.add("JLD")
# Pkg.add("HDF5")
# Pkg.add("PyCallJLD")

In [2]:
# Import libraries
using Flux
using Flux.Losses
using DelimitedFiles
using Statistics
using Random
using ScikitLearn
using RDatasets
using FeatureSelectors
using JLD
using PyCallJLD

In [3]:
# Import ScikitLearn models
@sk_import svm:SVC
@sk_import tree:DecisionTreeClassifier
@sk_import linear_model:LogisticRegression
@sk_import neighbors: KNeighborsClassifier
@sk_import naive_bayes:GaussianNB 
@sk_import ensemble:VotingClassifier
@sk_import ensemble:StackingClassifier
@sk_import ensemble:BaggingClassifier
@sk_import decomposition:PCA

PyObject <class 'sklearn.decomposition._pca.PCA'>

In [4]:
# Constants
# Execute model test set
RUN_ANN_TEST = false
RUN_SVM_TEST = false
RUN_DT_TEST = false
RUN_KNN_TEST = false
ANN_FILE_PATH = "dataset/models/ann.jld"
SVM_FILE_PATH = "dataset/models/svm.jld"
DT_FILE_PATH = "dataset/models/dt.jld"
KNN_FILE_PATH = "dataset/models/knn.jld"

"dataset/models/knn.jld"

In [5]:
# Include the code done in previous practices
include("utils/practices_code.jl")
# Class that handle the data processing
include("utils/data_handler.jl")
# Class that handle the model processing
include("utils/model_handler.jl")

loadModel (generic function with 1 method)

# Data preprocessing

This section contains the preprocessing of the data

In [6]:
# Load the dataset and normalize
dataset = readdlm("dataset/clean_music_genre.csv",',');

println("Dataset original size: ", size(dataset))
println("Sample of original dataset: ", dataset[2,1:17])

# Remove line of headers, artist and song name. Separate train_x and train_y
train_x = dataset[2:size(dataset,1),3:16]
train_y = dataset[2:size(dataset,1),17]

# Remove loudness column
# train_x = train_x[:,1:end .!= 5]

# train_x = abs.(train_x)

# Number of classes (total)
# Number of classes (balanced)

# normalized_inputs = normalizeMinMax!(train_x) --> must be after splitting
# binary_outputs = oneHotEncoding(train_y) --> Done only for Ann on training

println("Inputs size: ", size(train_x))
println("Sample of inputs: ", train_x[1,:])
println("Outputs size: ", size(train_y))
println("Sample of Outputs: ", train_y[1])

Dataset original size: (17972, 17)
Sample of original dataset: Any["Bruno Mars", "That's What I Like (feat. Gucci Mane)", 52.2, 0.854, 0.564, 1.0, -8.630403119526399, 1, 0.03404, 0.0171, 0.00965882, 0.0849, 0.899, 134.071, 234.596, 4, 5]
Inputs size: (17971, 14)
Sample of inputs: Real[52.2, 0.854, 0.564, 1.0, 8.630403119526399, 1, 0.03404, 0.0171, 0.00965882, 0.0849, 0.899, 134.071, 234.596, 4]
Outputs size: (17971,)
Sample of Outputs: 5


In [7]:
# Using Hold Out function to split dataset into train and test
indexs = holdOut(size(train_x,1),0.2)

train_input = train_x[indexs[1],:]
train_output = vec(train_y[indexs[1],:])

test_input = train_x[indexs[2],:]
test_output = vec(train_y[indexs[2],:])

#normalization after splitting, so test data cannot affect the train data and the first touch between them should be in predictions.
train_input = normalizeMinMax!(train_input)
test_input = normalizeMinMax!(test_input)

println("Size original input data: ", size(train_x))
println("Size original output data: ", size(train_y))

println("Size train input data: ", size(train_input))
println("Size train output data: ", size(train_output))

println("Size test input data: ", size(test_input))
println("Size test output data: ", size(test_output))

println("Sample original input data: ", train_x[1,:])
println("Sample train input data: ", train_input[1,:])
println("Sample test input data: ", test_input[1,:])

Size original input data: (17971, 14)
Size original output data: (17971,)
Size train input data: (14377, 14)
Size train output data: (14377,)
Size test input data: (3594, 14)
Size test output data: (3594,)
Sample original input data: Real[52.2, 0.854, 0.564, 1.0, 8.630403119526399, 1, 0.03404, 0.0171, 0.00965882, 0.0849, 0.899, 134.071, 234.596, 4]
Sample train input data: [0.3645833333333333, 0.935442220787605, 0.6266190394059059, 0.3, 0.26251335945849663, 1.0, 0.07726161369193155, 0.04849397590361446, 0.00014017539446232097, 0.1218637992831541, 0.8687609796424511, 0.4855961240479257, 0.10945638869159291, 0.75]
Sample test input data: [0.4329896907216495, 0.2831819189009528, 0.4550736251627768, 0.4, 0.38778462112728623, 0.0, 0.05503685503685504, 0.6395582329317269, 1.301628679155151e-6, 0.14336868199972455, 0.330893883951908, 0.4933398946461923, 0.10401569074701209, 0.75]


In [8]:
# applying PCA, 
pca = PCA(0.95)
fit!(pca, train_input)

pca_train = pca.transform(train_input)
pca_test = pca.transform(test_input)

println("Train Patterns ", size(train_input), " -> ", size(pca_train))
println("Test Patterns ", size(test_input), " -> ", size(pca_test))

# PCA based on 95% variance, suggests that there are 5 features have noise and should be eliminated
# from the input data, it's worth to try apply it and compare, maybe after finding the optimaal
# parameters of the model

Train Patterns (14377, 13) -> (14377, 10)
Test Patterns (3594, 13) -> (3594, 10)


In [9]:
# Feature Selection:
# this function can be used to the most K important features after normalization and splitting.
# for optimal number of K, I am still searching
#FeatureSelection(train_x,train_y,k)

# Model experimentation

This section contains all experimentation of the models

In [10]:
indexs = crossvalidation(train_output, 10)
kFoldIndices = convert(Vector{Int64}, indexs)

println(size(kFoldIndices))

(14377,)


In [11]:
if RUN_ANN_TEST
    test_ANN_Model(train_input, train_output, test_input, test_output, kFoldIndices, ANN_FILE_PATH)
end

In [12]:
if RUN_SVM_TEST
    test_SVM_Model(train_input, train_output, test_input, test_output, kFoldIndices, SVM_FILE_PATH)
end

In [13]:
if RUN_DT_TEST
    test_DT_Model(train_input, train_output, test_input, test_output, kFoldIndices, DT_FILE_PATH)
end

In [14]:
if RUN_KNN_TEST
    test_KNN_Model(train_input, train_output, test_input, test_output, kFoldIndices, KNN_FILE_PATH)
end

Test results for KNN model: 
Parameters: Dict{Any, Any}("n_neighbors" => 3, "metric" => "nan_euclidean", "weights" => "uniform") Accuracy: 0.25707846339618273 Fscore: 0.2643538076626878
Parameters: Dict{Any, Any}("n_neighbors" => 2, "metric" => "nan_euclidean", "weights" => "uniform") Accuracy: 0.22786128619640236 Fscore: 0.24344234493612685
Parameters: Dict{Any, Any}("n_neighbors" => 1, "metric" => "nan_euclidean", "weights" => "uniform") Accuracy: 0.25338394582905605 Fscore: 0.24713451321470625
Parameters: Dict{Any, Any}("n_neighbors" => 4, "metric" => "nan_euclidean", "weights" => "uniform") Accuracy: 0.29337780546042935 Fscore: 0.28270154606708114
Parameters: Dict{Any, Any}("n_neighbors" => 5, "metric" => "nan_euclidean", "weights" => "uniform") Accuracy: 0.3109092855614183 Fscore: 0.2947446195839709
Parameters: Dict{Any, Any}("n_neighbors" => 6, "metric" => "nan_euclidean", "weights" => "uniform") Accuracy: 0.32204321896547017 Fscore: 0.30260049011048407
Parameters: Dict{Any, Any}

In [15]:
# Get the best models
#best_ANN = loadModel(ANN_FILE_PATH)
#if isnothing(best_ANN)
#    best_ANN = get_Best_ANN(train_input, train_output, kFoldIndices)
#end
best_SVM = loadModel(DT_FILE_PATH)
if isnothing(best_SVM)
    best_SVM = get_Best_SVM(train_input, train_output, kFoldIndices)
end
best_DT = loadModel(DT_FILE_PATH)
if isnothing(best_DT)
    best_DT = get_Best_DT(train_input, train_output, kFoldIndices)
end
best_KNN = loadModel(KNN_FILE_PATH)
if isnothing(best_KNN)
    best_KNN = get_Best_KNN(train_input, train_output, kFoldIndices)
end

In [16]:
#Define the models to train

#=models = Dict( "SVM" => SVC(probability=true), 
         "LR" =>LogisticRegression(),
         "DT"=> DecisionTreeClassifier(max_depth=4),
         "NB"=> GaussianNB())

base_models =  [ name for name in keys(models)]=#

In [17]:
# Perform the training for each model and calculate the test values (accuracy)
#=for key in keys(models)
    model = models[key]
    fit!(model,train_input, train_output)
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end=#

In [18]:
#Define the metaclassifier based on the base_models
#=models["Ensemble (Hard Voting)"] = VotingClassifier(estimators = [(name,models[name]) for name in base_models], 
                                                   n_jobs=-1)
fit!(models["Ensemble (Hard Voting)"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end=#

In [19]:
#=models["Ensemble (Soft Voting)"] = VotingClassifier(estimators = [(name,models[name]) for name in base_models], 
                                                   n_jobs=-1, voting="soft",weights=[1,2,2,1])
fit!(models["Ensemble (Soft Voting)"],train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,train_input, train_output)
    println("$key: $(acc*100) %")
end=#

In [20]:
#=models["Ensemble (Stacking)"] = StackingClassifier(estimators=[(name,models[name]) for name in base_models],
    final_estimator=SVC(probability=true), n_jobs=-1)
fit!(models["Ensemble (Stacking)"], train_input, train_output)=#

In [21]:
#=for key in keys(models)
    model = models[key]
    acc = score(model,train_input, train_output)
    println("$key: $(acc*100) %")
end=#

In [22]:
#=models["Bagging (SVC)"] = BaggingClassifier(base_estimator=SVC(),n_estimators=10, max_samples=0.50, n_jobs=-1)
fit!(models["Bagging (SVC)"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,train_input, train_output)
    println("$key: $(acc*100) %")
end=#

In [23]:
#=@sk_import ensemble:(AdaBoostClassifier, GradientBoostingClassifier)

models["Ada"] = AdaBoostClassifier(n_estimators=30)
fit!(models["Ada"], train_input, train_output)

models["GTB"] = GradientBoostingClassifier(n_estimators=30, learning_rate=1.0, max_depth=2, random_state=0)
fit!(models["GTB"], train_input, train_output)

for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end=#


In [24]:
#=@sk_import ensemble:RandomForestClassifier

models["RF"] = RandomForestClassifier(n_estimators=8, max_depth=nothing,
                                    min_samples_split=2, n_jobs=-1)
fit!(models["RF"], train_input, train_output)
    
for key in keys(models)
    model = models[key]
    acc = score(model,test_input, test_output)
    println("$key: $(acc*100) %")
end=#

In [25]:
#=p = bar(y=1:60,models["RF"].feature_importances_, orientation=:horizontal, legend = false)
xlabel!(p,"Gini Gain")
ylabel!(p,"Fearure")
title!("Feature Importance")=#

In [26]:
#=using Pkg;
Pkg.add("XGBoost")=#

In [27]:
#=using XGBoost;

train_output_asNumber= Vector{Number}(train_output);

@assert train_output_asNumber isa Vector{Number}=#

In [28]:
#model = xgboost(train_input, 20, label = train_output_asNumber, eta = 1, max_depth = 6)

In [29]:
#=param = ["max_depth" => 2,
         "eta" => 1,
         "objective" => "binary:logistic"]
metrics = metrics = ["error", "auc"]
model = xgboost(train_input, 20, label = train_output_asNumber, param = param, metrics = metrics)

pred = predict(model, train_input)=#

In [30]:
#=using XGBoost: predict as predict_xgb

pred = predict_xgb(model, test_input)
print("Error of XGboost= ", sum((pred .> 0.5) .!= test_output) / float(size(pred)[1]), "\n")=#

In [31]:
#=feature_gain = map(x-> (x.fname,x.gain), importance(model))
feature, gain = first.(feature_gain), last.(feature_gain)

using Plots;

p = bar(gain, y=feature, orientation="h", legend=false)
xlabel!(p,"Gain")
ylabel!(p,"Feature")
title!("Feature Importance")=#